# Übungszettel 1 - Grundlagen, Resourcen, Annotationen
### Abgabefrist: Freitag, 28. April 2023, 9:59 Uhr

_Umbennenung des Jupyter Notebooks mit Gruppenname und Techfak-Kürzel!_

Punkte insgesamt möglich: _20 Punkte_

## Aufgabe 1: _(5 Punkte)_
Dem Übunszettel ist ein Ausschnitt aus dem Film "2001: A Space Odyssey" (`2001.mp4`) und ein Video `office-scene.mp4` beigelegt. Speichert diese im gleichen Verzeichnis wie das Jupyter Notebook oder schaut euch diese in einem eigenen Player an.

In [ ]:
%%HTML
<center>
<video width="640" height="480" controls>
  <source src="./2001.mp4" type="video/mp4">
</video>
<video width="640" height="480" controls>
  <source src="./office-scene.mp4" type="video/mp4">
</video>
</center>

### A1.1: _(3 Punkte)_
Überlegt welche Fähigkeiten und Wissen über die natürlichsprachliche Kommunikation der Computer HAL implementieren muss, um die dargestellte Kommunikationsepisode zu ermöglichen. Nennt mindestens sechs Punkte.

#### Antwort:
    <hier Lösung der Teilaufgabe 1.1 (Enter oder Doppelklick auf Zelle)>

### A1.2: _(2 Punkte)_
Erklärt den Begriff "Overlap" in der natürlichsprachlichen Kommunikation. Ergänzt eure Erläuterung um ein Beispiel aus dem Video `office-scene.mp4`.

#### Antwort:
    <hier Lösung der Teilaufgabe 1.2>

## Aufgabe 2: _(12 Punkte)_
Für komplexe Annotationen von Sprach- sowie Gestendaten wird im Allgemeinen Software wie [Praat](https://www.fon.hum.uva.nl/praat/) und [ELAN](https://archive.mpi.nl/tla/elan/download) verwendet. Für die naive Transkribierung soll hier das Tool [Label Studio](https://labelstud.io/) genügen.
Installiert dieses mittels

In [ ]:
!pip install label-studio

Startet die Label Studio Instanz und öffnet den erschienenden Link _(http://0.0.0.0:8080/)_:

In [ ]:
!label-studio

Gebt eine Zeichenreihenfolge im `Sign up`-Reiter ein, welche dem Pattern einer Email-Adresse entspricht und ein Passwort (test@test.de, 123456789). Gegebenfalls loggt euch mit den Daten ein.

### A2.1: _(8 Punkte)_
Transkribiert die das Gespräch aus der `office-scene.mp4`. Benutzt hierfür die `office-scene.wav` Audiodatei.
1. Legt dafür ein **neues Projekt** in Label Studio an.

    a) Wählt einen Namen für das Projekt.
    
    b) **Importiert** die WAV-Datei.
    
    c) Erstellt ein **custom template** mit folgendem HTML-Code:

    <View>
      <Labels name="labels" toName="audio">
        <Label value="Hendrik" />
        <Label value="Christopher" />
      </Labels>
      <AudioPlus name="audio" value="$audio"/>
      <TextArea name="transcription" toName="audio"
                rows="2" editable="true"
                perRegion="true" required="true" />
    </View>

2. Transkribiert den Dialog wortwörtlich (in deutscher Sprache) und in der Granularität von Äußerungen. **Hendrik** ist die Person, die zuerst spricht. Man kann mit `Strg + Scrollen` in der Darstellung der Audiodatei zoomen. Versucht für die Aufgabe 3 möglicht genau Start und Ende der Äußerungen zu markieren.
  
  _Hinweis:_ Wie in der Vorlesung bereits erörtert, gilt die Spracherkennung als "gelöstes" Problem. Qualitativ gibt es allerdings deutliche Unterschiede zwischen Deutsch und Englisch. Als eines der besten (und frei verfügbaren) Modelle gilt **Whisper** von OpenAI. Als _Entwurf_ für eure Transkriptionstexte bietet sich ein solches Modell an. [Whisper.cpp](https://github.com/ggerganov/whisper.cpp) ermöglicht es das Modell optimiert auf der eigenen CPU laufen zu lassen. Generell gibt es auch [Webseiten](https://replicate.com/openai/whisper), die kostenlos das Modell laufen lassen. Ansonsten kann man auch ohne solche Modelle das 1-minütige Gespräch transkribieren.
  
3. In Label Studio: `Submit` eure Transkription und exportiert in der Übersicht des Projekts eure Annotationen als JSON Datei.
4. **Stopt die Ausführung** von Label Studio im Jupyter Notebook (ggf. durch "Interrupt Kernel ■").
6. **Fügt** eure Annotationen als JSON Code **unten ein**.

In [ ]:
import json

# Antwort hier
transcript = json.loads("""
<ersetz mich>
""")

In [ ]:
def postprocessing(annotations):
    """Combine speaker and text annotation."""
    output = []
    current_sentence = {}
    for annotations_result in annotations: # annotation sessions
        for result in annotations_result["result"]:
            assert "value" in result, "Wrong format"
            assert "id" not in current_sentence or result["value"]["id"] == current_sentence["id"], "Wrong ordering in annotation"
            current_sentence.update(result["value"])
            if "labels" in current_sentence and "text" in current_sentence:
                current_sentence["labels"] = ",".join(current_sentence["labels"])  # for multiple labels 
                current_sentence["text"] = ";".join(current_sentence["text"])  # for multiple text annotations
                current_sentence["start"] = int(current_sentence["start"]*1000)  # to ms
                current_sentence["end"] = int(current_sentence["end"]*1000)  # to ms
                output.append(current_sentence)
                current_sentence = {}
    return output
transcript = postprocessing(transcript[0]["annotations"])

In [ ]:
!pip install plotly
!pip install pandas

In [ ]:
import pandas as pd
import plotly.express as px

def convert_ms_to_datetime(data):
    for sentence in data:
        sentence["start_dt"] = pd.to_datetime(sentence["start"], unit="ms")
        sentence["end_dt"] = pd.to_datetime(sentence["end"], unit="ms")
    return data

# Plot Transcript
figure = px.timeline(convert_ms_to_datetime(transcript), y="labels", color="labels", x_start='start_dt', x_end='end_dt', hover_data=["labels", "text"], text="text")
figure.update_layout(xaxis=dict(tickformat="%M:%S.%f"), xaxis_title="Zeit", yaxis_title="Sprecher")
figure.show()

### A2.2: _(1 Punkt)_
Welche Information **fehlen** im Vergleich zu einem GAT-2 Basistranskript bzw. Feintranskript in eurem Transkription?

#### Antwort:
    <hier Lösung der Teilaufgabe 2.3>

### A2.4: _(3 Punkt)_
Berechnet den **Floor Transfer Offset** (zeitlichen Abstand zwischen den Sprechwechseln / Floor Übergabe) für euere Transkription. Zwischenbemerkungen (kompletter Overlaps) sind dabei zu ignorieren.

In [ ]:
def floor_transfer_offset(transcript_data):
    transfer_offsets = []  # list of offsets (ms)
    
    # Antwort hier  
    
    return transfer_offsets

offsets = floor_transfer_offset(transcript)

print(offsets)
figure = px.histogram(offsets).update_layout(xaxis_title="milliseconds overlap")
figure.show()

## Aufgabe 3: _(3 Punkte)_

### A3.1: _(1 Punkt)_
Wofür wird die **Cohen's Kappa** Metrik verwendet? Wie wird sie berechnet für zwei Annotationen mit einer variablen Anzahl an Kategorien $C$ (Formel) und wie werden deren Bestandteile berechnet (Formel)? Markdown unterstützt Latex Mathe Formeln.

#### Antwort:
    Zweck: <Antwort>

    Formel: 
$$\kappa = $$
    
$$...$$
    
$$...$$
    
$$...$$

### A3.2: _(1 Punkt)_:
Berechnet für folgende Annotationen Cohen's Kappa (mit zwischen Ergebnissen). Entweder per Hand (Antwortzelle in Markdown ändern + Latex Umgebung mit $) oder per Python:

In [ ]:
from IPython.display import HTML, display

data = {"matrix": [[12, 2, 5, 0], [2, 13, 2, 0], [0, 1, 9, 2], [2, 1, 4, 10]], "categories": ["C1", "C2", "C3", "C4"]}

display(HTML("<table><tr><th>A1/A2</th><th>{}</th></tr><tr>{}</tr></table>".format("</th><th>".join(data["categories"]), "</tr><tr>".join(f"<th>{data['categories'][i]}</th><td>{'</td><td>'.join(str(entry) for entry in row)}</td>" for i, row in enumerate(data['matrix'])))))

In [ ]:
#### Antwort:

print("Zwischenergebnisse:", "")
print("Cohen's Kappa Wert:", "")


### A3.3: _(1 Punkt)_
Wie kann man den Wert aus A3.2 interpretieren?

#### Antwort:
    <Antwort>

------
Geben Sie dieses umbenannte und bearbeitete Jupyter Notebook im Moodle ab.